# How to add summary of the conversation history

One of the most common use cases for persistence is to use it to keep track of conversation history. This is great - it makes it easy to continue conversations. As conversations get longer and longer, however, this conversation history can build up and take up more and more of the context window. This can often be undesirable as it leads to more expensive and longer calls to the LLM, and potentially ones that error. One way to work around that is to create a summary of the conversation to date, and use that with the past N messages. This guide will go through an example of how to do that.

This will involve a few steps:

- Check if the conversation is too long (can be done by checking number of messages or length of messages)
- If yes, the create summary (will need a prompt for this)
- Then remove all except the last N messages

A big part of this is deleting old messages. For an in depth guide on how to do that, see [this guide](../delete-messages)

## Setup

First, let's set up the packages we're going to want to use

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic

Next, we need to set API keys for Anthropic (the LLM we will use)

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

ANTHROPIC_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

## Build the chatbot

Let's now build the chatbot.

In [3]:
from typing import Literal

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage
from langgraph.checkpoint.redis import RedisSaver
from langgraph.graph import MessagesState, StateGraph, START, END

# Set up Redis connection for checkpointer
REDIS_URI = "redis://redis:6379"
memory = None
with RedisSaver.from_conn_string(REDIS_URI) as cp:
    cp.setup()
    memory = cp


# We will add a `summary` attribute (in addition to `messages` key,
# which MessagesState already has)
class State(MessagesState):
    summary: str


# We will use this model for both the conversation and the summarization
model = ChatAnthropic(model_name="claude-3-haiku-20240307")


# Define the logic to call the model
def call_model(state: State):
    # If a summary exists, we add this in as a system message
    summary = state.get("summary", "")
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# We now define the logic for determining whether to end or summarize the conversation
def should_continue(state: State) -> Literal["summarize_conversation", END]:
    """Return the next node to execute."""
    messages = state["messages"]
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    # Otherwise we can just end
    return END


def summarize_conversation(state: State):
    # First, we summarize the conversation
    summary = state.get("summary", "")
    if summary:
        # If a summary already exists, we use a different system prompt
        # to summarize it than if one didn't
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # We now need to delete messages that we no longer want to show up
    # I will delete all but the last two messages, but you can change this
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}


# Define a new graph
workflow = StateGraph(State)

# Define the conversation node and the summarize node
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `conversation`.
    # This means these are the edges taken after the `conversation` node is called.
    "conversation",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `summarize_conversation` to END.
# This means that after `summarize_conversation` is called, we end.
workflow.add_edge("summarize_conversation", END)

# Finally, we compile it!
app = workflow.compile(checkpointer=memory)

## Using the graph

In [4]:
def print_update(update):
    for k, v in update.items():
        for m in v["messages"]:
            m.pretty_print()
        if "summary" in v:
            print(v["summary"])

In [5]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="hi! I'm bob")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="i like the celtics!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hi Bob! It's nice to meet you. I'm Claude, an AI assistant created by Anthropic. I'm here to help out however I can. Please let me know if you have any questions or if there's anything I can assist you with.
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

You said your name is Bob, so that is the name I have for you.
================================ Human Message =================================

i like the celtics!
================================== Ai Message ==================================

That's great that you're a Celtics fan! The Celtics are a storied NBA franchise with a rich history of success. Some key things about the Celtics:

- They have won 17 NBA championships, the

We can see that so far no summarization has happened - this is because there are only six messages in the list.

In [6]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='6bb57452-d968-4ca2-b641-a72a09b7dfbf'),
  AIMessage(content="Hi Bob! It's nice to meet you. I'm Claude, an AI assistant created by Anthropic. I'm here to help out however I can. Please let me know if you have any questions or if there's anything I can assist you with.", additional_kwargs={}, response_metadata={'id': 'msg_011jBGcbsvqnA6gCXExmN1a6', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 12, 'output_tokens': 56}, 'model_name': 'claude-3-haiku-20240307'}, id='run-39f0f967-454c-4047-a3db-9196c041668b-0', usage_metadata={'input_tokens': 12, 'output_tokens': 56, 'total_tokens': 68, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}),
  HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='5fd5c63c-f680-45c9-ba74-ae3

Now let's send another message in

In [7]:
input_message = HumanMessage(content="i like how much they win")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

i like how much they win
================================== Ai Message ==================================

I agree, the Celtics' consistent winning over the decades is really impressive. A few reasons why the Celtics have been so successful:

- Great coaching - They've had legendary coaches like Red Auerbach, Doc Rivers, and now Ime Udoka who have gotten the most out of their talented rosters.

- Sustained excellence - Unlike some teams that have short windows of success, the Celtics have been a perennial contender for the majority of their history.

- Ability to reload - Even when they lose star players, the Celtics have done a great job of rebuilding and restocking their roster to remain competitive.

- Knack for developing talent - Players like Larry Bird, Kevin McHale, and others have blossomed into all-time greats under the Celtics' system.

The Celtics' winning culture and pedigree as an organizatio

If we check the state now, we can see that we have a summary of the conversation, as well as the last two messages

In [8]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='i like how much they win', additional_kwargs={}, response_metadata={}, id='26916ba3-a474-48ec-a3d2-0da1d3b9f433'),
  AIMessage(content="I agree, the Celtics' consistent winning over the decades is really impressive. A few reasons why the Celtics have been so successful:\n\n- Great coaching - They've had legendary coaches like Red Auerbach, Doc Rivers, and now Ime Udoka who have gotten the most out of their talented rosters.\n\n- Sustained excellence - Unlike some teams that have short windows of success, the Celtics have been a perennial contender for the majority of their history.\n\n- Ability to reload - Even when they lose star players, the Celtics have done a great job of rebuilding and restocking their roster to remain competitive.\n\n- Knack for developing talent - Players like Larry Bird, Kevin McHale, and others have blossomed into all-time greats under the Celtics' system.\n\nThe Celtics' winning culture and pedigree as an organization is re

We can now resume having a conversation! Note that even though we only have the last two messages, we can still ask it questions about things mentioned earlier in the conversation (because we summarized those)

In [9]:
input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

You haven't explicitly told me your name in our conversation, so I don't know what your name is. I addressed you as "Bob" earlier based on the context, but I don't have definitive information about your actual name. If you let me know your name, I'll be happy to refer to you by it going forward.


In [10]:
input_message = HumanMessage(content="what NFL team do you think I like?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

what NFL team do you think I like?
================================== Ai Message ==================================

Hmm, without any additional information about your preferences, it's hard for me to confidently guess which NFL team you might like. There are so many great NFL franchises, each with their own passionate fanbases. 

Since we've been discussing your interest in the Boston Celtics, one possibility could be that you're a fan of another New England team, like the Patriots. Their success over the past couple of decades has certainly earned them a large and devoted following.

Alternatively, you could be a fan of a team with a strong connection to basketball, like the Dallas Cowboys which play in the same stadium as the NBA's Mavericks.

Or you might support an underdog team that's been on the rise, like the Cincinnati Bengals or Jacksonville Jaguars, who have developed exciting young cores.

Rea

In [11]:
input_message = HumanMessage(content="i like the patriots!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

i like the patriots!
================================== Ai Message ==================================

Ah I see, that makes a lot of sense! As a fellow Boston sports fan, it's great to hear that you're also a supporter of the New England Patriots.

The Patriots have been one of the most dominant and consistent franchises in the NFL over the past two decades, with 6 Super Bowl championships during the Tom Brady and Bill Belichick era. Their sustained excellence and championship pedigree is really impressive.

Some of the things that make the Patriots such an appealing team to root for:

- Winning culture and high expectations year after year
- Innovative, adaptable game-planning and coaching from Belichick
- Clutch performances from legendary players like Brady, Gronkowski, etc.
- Passionate, loyal fanbase in the New England region

It's always fun to be a fan of a team that is consistently in contention f